<a href="https://www.kaggle.com/code/taanieluleksin/milp-implementation-of-permutation-puzzles?scriptVersionId=252009847" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# Run this first!
import numpy as np
import pandas as pd
import json

# MILP implementation of permutation puzzles
This notebook shows how to implement such permutation puzzles as mixed-integer linear program - MILP (or even integer programming - IP) models in OR-tools and GAMS (which can be fed to commercial solvers like CPLEX and Gurobi). As an example we will use first puzzle only. As the search space is very huge, MILP is not the best way to tackle these puzzles and therefore it takes an eternity to solve even the smaller puzzles. Also a major setback is that limit for the maximum number of moves needs to be defined.

# MILP formulation of permutation puzzle
In this section, the mathematical formulation of the puzzle is described. This is mostly inspired by [article: "God’s Algorithm for Rubik’s Cube An Integer Programming Approach"](https://www.m-hikari.com/imf-password2009/45-48-2009/aksopIMF45-48-2009-2.pdf), but a bit generalized using [big-M method](https://en.wikipedia.org/wiki/Big_M_method) and introducing wildcards.

## Inputs
 - $N$ - number of different colors;
 - $x^{initial}_i$ - initial state of puzzle;
 - $x^{final}_i$ - final state of puzzle;
 - $N_w$ - number of allowed wildcards;
 - $T$ - maximum number of moves allowed;
 - $G$ - set of movements for puzzle, which indicate that move $k$ will move face $i$ to position $j$.

## Variables
 - $y_{t,k} \in \{0,1\}$ - determines if a move $k$ is made on time $t$;
 - $x_{t,i} \in \mathbb{R}[0,N]$ - Face $i$ color at time $t$;
 - $w_i \in \{0,1\}$ - Determines if face $i$ is considered to be a wildcard.

## Constraints
Initial state of puzzle
$$x_{0,i} = x^{initial}_i \quad \forall i $$

Final state of puzzle with wildcards
$$x_{T-1,i} - N \cdot w_i \leq x^{final}_i \leq x_{T-1,i} + N \cdot w_i \quad \forall i $$

Allowed number of wildcards
$$\sum_i w_i \leq N_w $$

**Movement constraints**

If a move is made, it will be according to defined movements
$$x_{t,i} - N \cdot (1-y_{t,k}) \leq x_{t+1,j} \leq x_{t,i} + N \cdot (1-y_{t,k}) \quad \forall k,i,j \in G, t$$

If a move is not made, faces will stay the same
$$x_{t,i} - N \cdot \sum_k y_{t,k} \leq x_{t+1,i} \leq x_{t,i} + N \cdot \sum_k y_{t,k} \quad \forall i,t$$

One move at a time
$$\sum_k y_{t,k} \leq 1 \quad \forall t$$

## Goal
Goal is to minimize the number of movements. Multiplier $t$ is applied to keep movements to the beginning.
$$\min \sum_{t,k} t \cdot y_{t,k}$$

# Let's load up the inputs for the models
This is taken from A* model notebooks.

In [2]:
general_path = '/kaggle/input/santa-2023/'
puzzle_info_path = general_path + 'puzzle_info.csv'
puzzles_path = general_path + 'puzzles.csv'
sample_submission_path = general_path + 'sample_submission.csv'

# Loading the data
puzzle_info_df = pd.read_csv(puzzle_info_path)
puzzles_df = pd.read_csv(puzzles_path)
sample_submission_df = pd.read_csv(sample_submission_path)

# Converting the string representation of allowed_moves to dictionary
puzzle_info_df['allowed_moves'] = puzzle_info_df['allowed_moves'].apply(lambda x: json.loads(x.replace("'", '"')))

# Selecting an example puzzle type and displaying its allowed moves
example_puzzle_type = puzzle_info_df['puzzle_type'].iloc[0]
example_allowed_moves = puzzle_info_df[puzzle_info_df['puzzle_type'] == example_puzzle_type]['allowed_moves'].iloc[0]

example_puzzle_type

'cube_2/2/2'

In [3]:
# Create values from solution state
# Map these values to initial and solution state
current_puzzle = puzzles_df.iloc[0]
unique_strings = set(current_puzzle['solution_state'].split(';'))
string_to_number = {string: index for index, string in enumerate(unique_strings, start=0)}
solution_array = [string_to_number[char] for char in current_puzzle["solution_state"].split(';') if char in string_to_number]
initial_array = [string_to_number[char] for char in current_puzzle["initial_state"].split(';') if char in string_to_number]
current_puzzle

id                                                              0
puzzle_type                                            cube_2/2/2
solution_state    A;A;A;A;B;B;B;B;C;C;C;C;D;D;D;D;E;E;E;E;F;F;F;F
initial_state     D;E;D;A;E;B;A;B;C;A;C;A;D;C;D;F;F;F;E;E;B;F;B;C
num_wildcards                                                   0
Name: 0, dtype: object

# Generation of inputs mentioned in the formulation

In [4]:
# Load movement indices for solving
# G is a list of tuples, which indicate that move k will move face i to position j
G = []
key_index = 0
move_names = list(example_allowed_moves.keys())
num_of_base_moves = len(move_names)
move_dict = {}
for key, value in example_allowed_moves.items():
    for i in range(len(value)):
        G.append((key_index, i, value[i]))
        #Add inverse move
        G.append((key_index + num_of_base_moves, value[i], i))
        #Generate move dictionary so the solution can be translated back to necessary format
        move_dict[key_index] = "-" + key
        move_dict[key_index+num_of_base_moves] = key
    key_index +=1

In [5]:
#Inputs
MaxMoves = 10
NumberOfColors = len(unique_strings)
NumberOfFaces = len(solution_array)
NumberOfMoves = num_of_base_moves*2

NumberOfWildcards = current_puzzle["num_wildcards"]

InitialState = initial_array
SolutionState = solution_array

# OR-tools implemenatation
Following is the OR-tools implementation of the generalized MILP formulation of permutation puzzle.

In [6]:
from ortools.linear_solver import pywraplp

In [7]:
#Note that this can be changed to "SCIP" as well. Feel free to play around!
model = pywraplp.Solver.CreateSolver("SAT")
# Decision variables
y = [] #Determines if move i is made on time t
x = [] #Determines face value on j on time t
for t in range(MaxMoves):
    y_i = [model.NumVar(0,1,f"y_{i}_{t}") for i in range(NumberOfMoves)]
    x_j = [model.NumVar(0, NumberOfColors-1, f"x_{j}_{t}") for j in range(NumberOfFaces)]
    y.append(y_i)
    x.append(x_j)

if NumberOfWildcards > 0: #Use wildcard variable only when necessary
    w = [model.NumVar(0, NumberOfColors-1, f"w_{j}") for j in range(NumberOfFaces)]

#Initial position constraints
for i in range(NumberOfFaces):
    model.Add(x[0][i] == InitialState[i])

#Movement constraints
for t in range(MaxMoves-1):
    #If movement is made, faces move according to movements.
    for k, i, j in G:
        #print(f"Indices k={k}, i = {i}, j = {j}, t = {t}")
        model.Add(x[t][i] - NumberOfColors*(1 - y[t][k]) <= x[t+1][j])
        model.Add(x[t+1][j] <= x[t][i] + NumberOfColors*(1 - y[t][k]))

    #Ensures that if no movement is made faces will stay the same.
    for i in range(NumberOfFaces):
        model.Add(x[t][i] - NumberOfColors*(model.Sum([y[t][l] for l in range(NumberOfMoves)])) <= x[t+1][i])
        model.Add(x[t+1][i] <= x[t][i] + NumberOfColors*(model.Sum([y[t][l] for l in range(NumberOfMoves)])))
    
#One move at a time
for t in range(MaxMoves):
    model.Add(model.Sum([y[t][k] for k in range(NumberOfMoves)]) <= 1)

#Final position constraints
if NumberOfWildcards > 0: #Use wildcard constraint only when necessary
    for i in range(NumberOfFaces):
        model.Add(x[MaxMoves-1][i] - NumberOfColors*w[i] <= SolutionState[i])
        model.Add(SolutionState[i] <= x[MaxMoves-1][i] + NumberOfColors*w[i])
    model.Add(model.Sum([w[i] for i in range(NumberOfFaces)]) <= NumberOfWildcards)
else:
    for i in range(NumberOfFaces):
        model.Add(x[MaxMoves-1][i] == SolutionState[i])

#Objective
model.Minimize(model.Sum([y[t][i]*t for t in range(MaxMoves) for i in range(NumberOfMoves)]))
model.set_time_limit(300*1000) #Limit time in ms
model.Solve()

0

## Summary of the solution

In [8]:
print(f"Objective value: {model.Objective().Value()}")
if NumberOfWildcards > 0:
    Wildcards = [w[i].solution_value() for i in range(NumberOfFaces)]
    print(f"Number of wrong faces {np.array(Wildcards).sum()}")
Faces = [[np.round(x[t][i].solution_value() + 0.0,0) for i in range(NumberOfFaces)] for t in range(MaxMoves)]
Moves = [[y[t][k].solution_value() for k in range(NumberOfMoves)] for t in range(MaxMoves)]
print(f"Number of moves: {np.array(Moves).sum()}")

Objective value: 1.0
Number of moves: 2.0


### Putting together solution in correct format

In [9]:
solution = []
for t in range(MaxMoves):
    for i in range(NumberOfMoves):
        if Moves[t][i] > 0:
            solution += [move_dict[i]]
solution_alg = '.'.join(solution)
print("Solution for puzzle:")
print(solution_alg)

Solution for puzzle:
r1.-f1


# GAMS implementation - for commercial solvers
Using the same principles above we can create a GAMS formulation of the problem, which can be fed to commercial solvers like CPLEX or Gurobi through [NEOS server](https://neos-server.org/neos/) ([link for CPLEX problem insertion](https://neos-server.org/neos/solvers/milp:CPLEX/GAMS.html)) for instance.

## Generation of inputs mentioned in the formulation for GAMS
Logic is similar as before. Just transforming to [GAMS](https://www.gams.com/) input format.

In [10]:
# Load movement indices for solving
key_index = 0
move_names = list(example_allowed_moves.keys())
num_of_base_moves = len(move_names)
GAMS_set = ""
for key, value in example_allowed_moves.items():
    for i in range(len(value)):
        GAMS_set = ', '.join(filter(None, [GAMS_set, f"{key_index+1}.{i+1}.{value[i]+1}"]))
        #Add inverse move
        GAMS_set = ', '.join(filter(None, [GAMS_set, f"{key_index + num_of_base_moves+1}.{value[i]+1}.{i+1}"]))
    key_index +=1

In [11]:
GAMS_initial_state = ""
GAMS_solution_state = ""
for i in range(NumberOfFaces):
    GAMS_initial_state = ', '.join(filter(None, [GAMS_initial_state, f"{i+1}    {InitialState[i]+1}"]))
    GAMS_solution_state = ', '.join(filter(None, [GAMS_solution_state, f"{i+1}    {SolutionState[i]+1}"]))

## Putting together GAMS formulation

In [12]:
GAMS_formulation = f'''Sets
    i /1*{NumberOfFaces}/
    t /1*{MaxMoves}/
    k /1*{NumberOfMoves}/;

Alias (j, i);
Alias (k, kk);
Alias (t, tt);
    
Set G(k,i,j) / {GAMS_set} /;

Parameters
    x_initial(i) / {GAMS_initial_state} /
    x_final(i) / {GAMS_solution_state} /;

Scalars
    N_w  / {NumberOfWildcards} /
    N  / {NumberOfColors+1} /;

Variable
    movements;

Positive variable
    x(t, i);

Binary variables
    y(t, k)
    w(i);

Equations
    initial_state(i)
    final_state_lower(i)
    final_state_upper(i)
    wildcard_limit
    movement_constraints_lower(t,k,i,j)
    movement_constraints_upper(t,k,i,j)
    stay_same_lower(t, i)
    stay_same_upper(t, i)
    one_move_at_a_time(t)
    total_movements;

* Initial state of puzzle constraint
initial_state(i).. x('1', i) =e= x_initial(i);

* Final state of puzzle with wildcards constraint
final_state_lower(i).. x('{MaxMoves}', i) - N*w(i) =l= x_final(i);
final_state_upper(i).. x_final(i) =l= x('{MaxMoves}', i) + N*w(i);

* Allowed number of wildcards constraint
wildcard_limit.. sum(i, w(i)) =l= N_w;

* Movement constraints if a move is made
movement_constraints_lower(t,k,i,j)$(G(k,i,j) and (ord(t) < card(t))).. 
    x(t, i) - N*(1 - y(t, k)) =l= x(t+1, j);
movement_constraints_upper(t,k,i,j)$(G(k,i,j) and (ord(t) < card(t))).. 
    x(t+1, j) =l= x(t, i) + N*(1 - y(t, k));

* If no move is made, faces will stay the same
stay_same_lower(t, i)$(ord(t) < card(t)).. 
    x(t, i) - N*sum(k, y(t, k)) =l= x(t+1, i);
stay_same_upper(t, i)$(ord(t) < card(t)).. 
    x(t+1, i) =l= x(t, i) + N*sum(k, y(t, k));

* One move at a time constraint
one_move_at_a_time(t).. sum(k, y(t, k)) =l= 1;

* Objective function: Minimize the number of movements
total_movements.. movements =e= sum((t, k), y(t, k)*ord(t));
Model perm_puzzle /all/;

solve perm_puzzle min movements use mip;
'''

In [13]:
with open("gams_formulation.gms", "w") as text_file:
    text_file.write(GAMS_formulation)

## Final words on GAMS
Since commercial solvers didn't perform necessarily better than ones provided with OR-tools, then I didn't create a translator for GAMS output to solution.